In [4]:
import os
import sys
import pandas as pd
import xray

import IPython.display as display

## Definitions of input parameters

### SW

<span style="color: green">LAY</span>

              LAY      layer number of aerosol layer.  (The layer numbering refers to the                                                                             
                       ordering for the upward radiative transfer, i.e. bottom to top.                                                                                
                       For IATM = 0 (Record 1.2), each layer\'s number is equal to the                                                                                
                       position of its Record 2.1.1 in the grouping of these records.                                                                                 
                       For example, the second Record 2.1.1 occurring after Record 2.1                                                                                
                       corresponds to the second layer.  For IATM = 1 (Record 1.2) and                                                                                
                       IBMAX > 0 (Record 3.1), layer n corresponds to the region between      -                                                                        
                       altitudes n and n+1 in the list of layer boundaries in Record 3.3B.                                                                            
                       For IATM = 1 (Record 1.2) and IBMAX = 0 (Record 3.1), the layer                                                                                
                       numbers can be determined by running RRTM for the cloudless case                                                                               
                       and examining the TAPE6 output from this run.(
                       
                  
                       

        
     
<span style="color: green">AERPAR</span>


      AERPAR  (only used if IAOD = 0) array of parameters for obtaining aerosol optical depth as a                                                                    
              function of wavelength, as described below:                                                                                                             
                                                                                                                                                                      
        AOD = AOD1 * (AERPAR(2) + AERPAR(3) * (lambda/lambda1)) /                                                                                                     
                       ((AERPAR(2) + AERPAR(3) - 1) + (lambda/lambda1)**AERPAR(1))                                                                                    
              where                                                                                                                                                   
                  lambda = wavelength in microns                                                                                                                      
                  lambda1 = 1 micron                                                                                                                                  
                  AOD = aerosol optical depth at wavelength lambda                                                                                                    
                  AOD1 = aerosol optical depth at 1 micron (see Record A2.1.1).                                                                                       
                                                                                                                                                                      
        This is a version of Eq. 13 from  Molineaux et al, Appl. Optics, 1998.  The default values of                                                                 
        AERPAR(1), AERPAR(2), and AERPAR(3), which are 0, 1, and 0, respectively, yield an aerosol                                                                    
        with spectrally grey extinction.                                                                                                                              
                                                                                                                                                                      
        (Note:  To obtain Angstrom relation, set AERPAR(2)=1., AERPAR(3)=0., and AERPAR(1) equal to                                                                   
        Angstrom exponent.)
                                

In [2]:
def aer_dir():
    return '/nuwa_data/data/cesm1/inputdata/atm/cam/physprops/'

def nc_files_read_in_physprop_init():
    print( '(file name, optics method, fileindex)' )
    filelist = [('mam3_mode1_rrtmg_c110318.nc', 'modal', 1),
                ('mam3_mode2_rrtmg_c110318.nc', 'modal', 2),
                ('mam3_mode3_rrtmg_c110318.nc', 'modal', 3),
                ('sulfate_rrtmg_c080918.nc', 'hygroscopic', 4),
                ('ocpho_rrtmg_c101112.nc', 'insoluble', 5),
                ('ocphi_rrtmg_c100508.nc', 'hygroscopic', 6),
                ('bcpho_rrtmg_c100508.nc', 'insoluble', 7),
                ('dust4_rrtmg_c090521.nc', 'insoluble', 8),
                ('ssam_rrtmg_c100508.nc', 'hygroscopic', 9)]
    return filelist

## Contents of cesm1/inputdata/atm/cam/physprops/bcpho_rrtmg_c100508.nc

In [6]:
ds = xray.open_dataset(os.path.join(aer_dir(), 'bcpho_rrtmg_c100508.nc'))

In [8]:
ds

<xray.Dataset>
Dimensions:                      (lw_band: 16, near-ir_band: 44, rh_idx: 1, sw_band: 14)
Coordinates:
  * lw_band                      (lw_band) int64 0 1 2 3 4 5 6 7 8 9 10 11 ...
  * near-ir_band                 (near-ir_band) int64 0 1 2 3 4 5 6 7 8 9 10 ...
  * rh_idx                       (rh_idx) int64 0
  * sw_band                      (sw_band) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13
Data variables:
    opticstype                   |S12 b'insoluble   '
    refindex_im_aer_lw           (lw_band) float64 -0.79 -0.79 -0.79 -0.79 ...
    dryrad                       float64 1.18e-08
    num_to_mass_ratio            float64 1.672e+19
    refindex_im_water_lw         (lw_band) float64 -0.336 -0.36 -0.4262 ...
    radmin_aer                   float64 6.606e-10
    abs_nir                      (near-ir_band, rh_idx) float64 4.576e+03 ...
    refindex_real_water_lw       (lw_band) float64 1.532 1.524 1.42 1.274 ...
    refindex_real_aer_sw         (sw_band) float64 1.95 1.9

# IN_AER_RRTM

Problems:

1. ssa and asy output by modal_aero_sw() have dimensions *layers* and *spectral bands*, but they only have dimensions *spectral bands* in IN_AER_RRTM.  What should be done about the *layers* dimension?

2. In modal_aero_sw(), the following variables from *model state* are required:

   1. layer mass, *mass(pcols, pver)*
   2. air density, *air_density(pcols, pver)*
   3. species mass mixing ratio, *specmmr(pcols, pver: m, l)*
   
3. In modal_aero_sw(), the following variables from physical buffer are required:

   1. number mode wet diameter for all modes, *dgnumwet_m(pcols, pver, nmodes)*
   2. aerosol water for all modes, *qaerwat_m(pcols, pver, nmodes)*
   3. species mass mixing ratio, *specmmr(pcols, pver: m, l)*

In [10]:
display.HTML('''
<script>
show_code = true;
function code_toggle() {
if (show_code) {$('div.input').hide();} else {$('div.input').show();}
show_code = !show_code
}
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code"></form>
''')